In [1]:
import pandas as pd
import numpy as np
np.random.seed(0)
import time
start = time.time()
import re
import nltk
import string

#scrapping
from urllib.request import urlopen
from bs4 import BeautifulSoup
import requests

# import StemmerFactory class
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

# create stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()

#term weighting tfidf
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer


In [2]:
stopword = open("stopword_list_tala.txt", "r")
stopwords = stopword.read().split("\n")
corpus = pd.read_csv("corpus2.csv")

In [3]:
#this function does tokenize, remove stopwords, and stemming
def get_clean_corpus(raw_corpus, stopwords):    
    clean_corpus = []
    token = []
    for index, item in enumerate(corpus['dokumen']):
        term = corpus['dokumen'][index].split(" ")
        
        #deleting url
        deleted_url = [temp for temp in term if not re.match(r"\w+(?:(\.(\w+)\.(\w+)))|\w+(?:(\.(\w+)))", str(temp))]
        
        #deleting symbol
        deleted_symbol = [re.sub(r"[\-\+\=\:\;\"\\\@\[\]\,_!;.':#$%^&*()<>?/\|}{~:]"," ",str(temp)) for temp in deleted_url ]
        
        #stemming
        stemmed_sentence = stemmer.stem(" ".join(deleted_symbol))
        
        tokens = stemmed_sentence.split(" ")
        
        for i in range(len(tokens)):
            for index, word in enumerate(tokens):
                #delete stopwprds
                if word in stopwords:
                    del tokens[index]
                    
                #delete number
                if word.isdigit():
                    del tokens[index]
        
        clean_corpus.append(" ".join(tokens))
        token.append(tokens)
        
    raw_corpus['clean_corpus'] = clean_corpus
    raw_corpus['token'] = token
    
    return raw_corpus

In [14]:
def get_term_weighting_score(corpus):
    #frequency
    vectorizer = CountVectorizer(min_df=0)
    print(corpus["clean_corpus"])
    print(corpus["clean_corpus"][29])
    freq_term_corpus = vectorizer.fit_transform(corpus["clean_corpus"]).toarray()
    
    #tf idf
    transformer = TfidfTransformer(norm=None, use_idf=True, smooth_idf=True,)
    tfidf = transformer.fit_transform(freq_term_corpus)
    
#     tfidf_vector = tfidf / np.linalg.norm(tfidf)
    
        
    indip_word =vectorizer.get_feature_names()
    print(transformer.idf_)
    print(len(transformer.idf_))
    print(vectorizer.get_feature_names())
    print(len(vectorizer.get_feature_names()))
    






    
    corpus['raw_freq'] = freq_term_corpus.tolist()
#     corpus['tfidf'] = tfidf_vector.tolist()
    return corpus, indip_word

In [5]:
result = get_clean_corpus(corpus, stopwords)
# result

In [15]:
result2 , indip_word= get_term_weighting_score(result)
result2

0     sekolah kota palembang sumatera selatan putus ...
1     kepala smp negeri palembang siti zubaida putus...
2     pagi pulang siswa lihat kabut asap tebal dampa...
3     amin kepala dinas didik kota palembang ahmad z...
4     tk smp negeri swasta derajat libur besok edar ...
5     kualitas udara palembang buruk giat ajar ajar ...
6     papar kepala dinas didik sumatera selatan wido...
7     daerah dampak kabut asap normal ajar daerah ka...
8     agus wibowo kepala pusat data informasi humas ...
9     pesan digital kepala dinas didik kota palemban...
10    warga palembang sumatera selatan keluh kabut a...
11    amelia warga lurah bukit sangkal camat kalidon...
12    kejut gelap kemarin kemarin kabut asap parah k...
13       keluh kondisi tina orang guru senam buah bugar
14    rumah wib jadwal senam kejut jarak pandang met...
15    kepala seksi observasi informasi stasiun meteo...
16                                   dampak bakar hutan
17    kabut asap kota palembang parah pek akibat

,dokumen,clean_corpus,token,raw_freq
0,"Sejumlah sekolah di Kota Palembang, Sumatera S...",sekolah kota palembang sumatera selatan putus ...,"[sekolah, kota, palembang, sumatera, selatan, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ..."
1,"Kepala SMP Negeri 7 Palembang, Siti Zubaida, m...",kepala smp negeri palembang siti zubaida putus...,"[kepala, smp, negeri, palembang, siti, zubaida...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,"""Pagi ini kami memulangkan siswa karena meliha...",pagi pulang siswa lihat kabut asap tebal dampa...,"[pagi, pulang, siswa, lihat, kabut, asap, teba...","[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ..."
3,Hal ini diamini Kepala Dinas Pendidikan Kota P...,amin kepala dinas didik kota palembang ahmad z...,"[amin, kepala, dinas, didik, kota, palembang, ...","[0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ..."
4,"""Hari ini seluruh TK hingga SMP negeri dan swa...",tk smp negeri swasta derajat libur besok edar ...,"[tk, smp, negeri, swasta, derajat, libur, beso...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
5,"Menurutnya, kalau kualitas udara Palembang mas...",kualitas udara palembang buruk giat ajar ajar ...,"[kualitas, udara, palembang, buruk, giat, ajar...","[0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
6,"Akan tetapi, sebagaimana dipaparkan Kepala Din...",papar kepala dinas didik sumatera selatan wido...,"[papar, kepala, dinas, didik, sumatera, selata...","[0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ..."
7,"""Daerah yang tidak terdampak kabut asap tetap ...",daerah dampak kabut asap normal ajar daerah ka...,"[daerah, dampak, kabut, asap, normal, ajar, da...","[0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ..."
8,"Hal itu belakangan dibenarkan Agus Wibowo, sel...",agus wibowo kepala pusat data informasi humas ...,"[agus, wibowo, kepala, pusat, data, informasi,...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
9,"""Melalui pesan digital, Kepala Dinas Pendidika...",pesan digital kepala dinas didik kota palemban...,"[pesan, digital, kepala, dinas, didik, kota, p...","[1, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [9]:
indip_word

['agus',
 'ahmad',
 'ajar',
 'akibat',
 'akses',
 'aktivitas',
 'amelia',
 'amin',
 'anak',
 'andriawan',
 'ansori',
 'arah',
 'asa',
 'asap',
 'atur',
 'badan',
 'bahaya',
 'bakar',
 'bambang',
 'barang',
 'batas',
 'bawa',
 'belah',
 'bencana',
 'beny',
 'berangkat',
 'berita',
 'besok',
 'bidang',
 'bnpb',
 'buah',
 'bugar',
 'bukit',
 'buruk',
 'camat',
 'campur',
 'capai',
 'daerah',
 'dampak',
 'darat',
 'daring',
 'darurat',
 'dasar',
 'data',
 'derajat',
 'dermaga',
 'didik',
 'digital',
 'dinas',
 'edar',
 'elshinta',
 'fokus',
 'gambut',
 'ganjil',
 'gelap',
 'genap',
 'gera',
 'giat',
 'golong',
 'grup',
 'guru',
 'hadap',
 'helikopter',
 'henti',
 'himbau',
 'humas',
 'hutan',
 'ii',
 'ilir',
 'informasi',
 'instruksi',
 'jadwal',
 'jam',
 'jarak',
 'jumat',
 'kabupaten',
 'kabut',
 'kadiknas',
 'kalidoni',
 'kantor',
 'kapal',
 'kategori',
 'kawasan',
 'kejut',
 'kelas',
 'keluh',
 'kemarin',
 'kepala',
 'kerah',
 'kirim',
 'komering',
 'kondisi',
 'kota',
 'ksop',
 'kuali

In [ ]:
# result2.to_csv(r'manual_til_raw_freq.csv')

In [ ]:
elapsed_time_fl = (time.time() - start) 
print(elapsed_time_fl)

In [ ]:
yap = stemmer.stem('www.ini_makam.co.id bukan juga 24/7 00.00 00,00ya!')
yap

In [ ]:

print(pd.__version__) 